In [1]:
# we need the following libraries, so let's install them
# %pip install boto3
# %pip install torchxrayvision
# %pip install SimpleITK
# %pip install radiomics
# %pip install pyradiomics
# %pip install torchvision
%pip install matplotlib
%pip install scikit-learn
%pip install tensorflow
%pip install imblearn

In [1]:
# importing libraries
from time import time
import multiprocessing
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
import skimage
from  imblearn.over_sampling import SMOTE
import torch
# import SimpleITK as sitk
import skimage, torch, torchvision
# import torchxrayvision as xrv
import torchvision
# import torchxrayvision as xrv
import matplotlib.pyplot as plt
import sys
import warnings
from sklearn.utils import shuffle
import random
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder
import logging
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore", category=DeprecationWarning)
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# loadnig the data and removing the useless column 'Unnamed: 0'
whole_data = pd.read_csv('/content/gdrive/MyDrive/radiomics/MIMIC_synced_data_nan_free_.csv')
# whole_data = pd.read_csv('MIMIC_synced_data_nan_free_.csv')
whole_data.head(5)

,Unnamed: 0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,0,-872.712177,-30.834509,1.183500e+10,5.213223,484.245094,2.123297,127.427505,259.288179,-535.029395,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,-860.588226,-94.596062,1.748392e+10,5.468412,406.884354,2.973349,455.105927,238.126282,-510.922325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,-771.608655,246.313766,1.054451e+10,5.639891,569.798054,2.284384,601.678467,308.126190,-357.526186,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,-918.119244,197.538876,1.041507e+10,5.650920,757.435298,1.774919,477.192200,369.400809,-432.636913,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,-809.137268,336.588287,1.613116e+10,5.824157,562.000023,2.493775,658.329407,330.944854,-337.573971,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [3]:
whole_data.iloc[:,464:]

,feature_463,index,path,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,...,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,0.000647,205043.0,132464 files/p15/p15434659/s54252684/79b44a...,15434659.0,54252684.0,132464 79b44a75-4947038f-b2032bf8-9f0964ba-...,132464 CHEST (PORTABLE AP)\nName: Performed...,"132464 AP\nName: ViewPosition, dtype: object",132464 CHEST (PORTABLE AP)\nName: Procedure...,132464 antero-posterior\nName: ViewCodeSequ...,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.000634,259684.0,167551 files/p16/p16897045/s57135195/98bc41...,16897045.0,57135195.0,167551 98bc415f-9177cc30-e397fa69-4549646c-...,167551 PORTABLE ABDOMEN\nName: PerformedPro...,"167551 AP\nName: ViewPosition, dtype: object",167551 CHEST (PORTABLE AP)\nName: Procedure...,167551 antero-posterior\nName: ViewCodeSequ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.000474,312046.0,201203 files/p18/p18270774/s52463788/59c3e9...,18270774.0,52463788.0,201203 59c3e983-4cf18980-f61334d7-d187daf4-...,201203 CHEST (PORTABLE AP)\nName: Performed...,"201203 AP\nName: ViewPosition, dtype: object",201203 CHEST (PORTABLE AP)\nName: Procedure...,201203 antero-posterior\nName: ViewCodeSequ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.001550,86825.0,56297 files/p12/p12317276/s54579430/d64ccc0...,12317276.0,54579430.0,56297 d64ccc05-412114b3-3d1abc31-44f0eaff-b...,56297 CHEST (PORTABLE AP)\nName: PerformedP...,"56297 AP\nName: ViewPosition, dtype: object",56297 CHEST (PORTABLE AP)\nName: ProcedureC...,56297 antero-posterior\nName: ViewCodeSeque...,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.000272,210911.0,136226 files/p15/p15581272/s57159127/2b9a79...,15581272.0,57159127.0,136226 2b9a793d-1cf1ba2b-dc1a0618-b5247e44-...,136226 CHEST (PORTABLE AP)\nName: Performed...,"136226 AP\nName: ViewPosition, dtype: object",136226 DX CHEST PORT LINE/TUBE PLCMT 2 EXAM...,136226 antero-posterior\nName: ViewCodeSequ...,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236698,0.000592,103309.0,66833 files/p12/p12748367/s57175290/1ce6533...,12748367.0,57175290.0,66833 1ce65331-1f90d760-8d950859-2257c659-c...,66833 CHEST (PORTABLE AP)\nName: PerformedP...,"66833 AP\nName: ViewPosition, dtype: object",66833 CHEST (PORTABLE AP)\nName: ProcedureC...,66833 antero-posterior\nName: ViewCodeSeque...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
236699,0.000499,109826.0,71013 files/p12/p12934243/s50773448/1aee4d3...,12934243.0,50773448.0,71013 1aee4d36-06dce1b8-d9a4bc11-db55b0a0-2...,71013 CHEST (PORTABLE AP)\nName: PerformedP...,"71013 AP\nName: ViewPosition, dtype: object",71013 CHEST (PORTABLE AP)\nName: ProcedureC...,71013 antero-posterior\nName: ViewCodeSeque...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236700,0.000306,135337.0,87467 files/p13/p13599784/s56578317/d3c4dcb...,13599784.0,56578317.0,87467 d3c4dcb9-a7d675ce-96d27d6b-939350fa-6...,87467 NaN\nName: PerformedProcedureStepDesc...,"87467 PA\nName: ViewPosition, dtype: object",87467 CHEST (PA AND LAT)\nName: ProcedureCo...,87467 postero-anterior\nName: ViewCodeSeque...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236701,0.000487,186180.0,120275 files/p14/p14959010/s54118053/731b58...,14959010.0,54118053.0,120275 731b58bc-7faa2b94-cea80a5f-afa26fad-...,120275 NaN\nName: PerformedProcedureStepDes...,"120275 PA\nName: ViewPosition, dtype: object",120275 CHEST (PA AND LAT)\nName: ProcedureC...,120275 postero-anterior\nName: ViewCodeSequ...,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
list(whole_data.iloc[:,464:])

['feature_463',
 'index',
 'path',
 'subject_id',
 'study_id',
 'dicom_id',
 'PerformedProcedureStepDescription',
 'ViewPosition',
 'ProcedureCodeSequence_CodeMeaning',
 'ViewCodeSequence_CodeMeaning',
 'Race',
 'Ethnicity',
 'Sex',
 'Age',
 'No Finding',
 'Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Enlarged Cardiomediastinum',
 'Fracture',
 'Lung Lesion',
 'Lung Opacity',
 'Pleural Effusion',
 'Pleural Other',
 'Pneumonia',
 'Pneumothorax',
 'Support Devices']

In [5]:
whole_data['Race']

0                  132464    NaN\nName: Race, dtype: object
1                167551    WHITE\nName: Race, dtype: object
2                201203    ASIAN\nName: Race, dtype: object
3                 56297    WHITE\nName: Race, dtype: object
4                136226    WHITE\nName: Race, dtype: object
                                ...                        
236698       66833    PORTUGUESE\nName: Race, dtype: object
236699            71013    WHITE\nName: Race, dtype: object
236700    87467    HISPANIC OR LATINO\nName: Race, dtype...
236701           120275    WHITE\nName: Race, dtype: object
236702            79530    WHITE\nName: Race, dtype: object
Name: Race, Length: 236703, dtype: object

In [6]:
2# excludiung Ages with Nans
temporary_labels = np.zeros((whole_data.shape[0],))
for i, val in enumerate(whole_data['Race'].values):
    if 'WHITE' in val:
        temporary_labels[i] = 0
    elif 'BLACK' in val or 'AFRICAN AMERICAN' in val or 'ASIAN' in val:
        temporary_labels[i] = 1
    else:
        temporary_labels[i] = np.nan


In [7]:
# the number of each label:
# white, african american, asian
(temporary_labels==0).sum(), (temporary_labels==1).sum()

(139608, 40996)

In [8]:
whole_data['new_race'] = temporary_labels.tolist()

In [9]:
# visualizing the data:
whole_data['new_race']

0         NaN
1         0.0
2         1.0
3         0.0
4         0.0
         ... 
236698    NaN
236699    0.0
236700    NaN
236701    0.0
236702    0.0
Name: new_race, Length: 236703, dtype: float64

In [10]:
whole_data = whole_data[~pd.isnull(whole_data['new_race'])]

In [11]:
whole_data.shape

(180604, 493)

In [12]:
random_seed = 21202
del whole_data['Unnamed: 0']
whole_data.head()
whole_data = whole_data.sample(frac=1.0, random_state=random_seed)

In [13]:
# finding total number of samples:
n_samples = whole_data.shape[0]
print("number of samples are: {}".format(n_samples))

number of samples are: 180604


In [14]:
# names of all targets:
targets_names = list(whole_data.iloc[:,464:])
print(targets_names)

['index', 'path', 'subject_id', 'study_id', 'dicom_id', 'PerformedProcedureStepDescription', 'ViewPosition', 'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning', 'Race', 'Ethnicity', 'Sex', 'Age', 'No Finding', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices', 'new_race']


In [15]:
# finding the number of unique subjects:
unique_subjects = pd.unique(whole_data['subject_id'])
print('The number of unique subjects are: {0}'.format(len(unique_subjects)))

The number of unique subjects are: 42654


In [16]:
# preallocating the frequency of session of the subjects:
# this is written to make sure that we dont use a subject in both training and testing split.
test_split = 0.06       # precent out of total samples
valid_split = 0.1       # precent out of total samples (this is the sumation of both training and validation splits)

n_test = n_samples * test_split
n_whole_train = n_samples - n_test

frequencies = np.ones((unique_subjects.shape[0], 4)) * np.nan
for i, sbj in tqdm(enumerate(unique_subjects)):
  frequencies[i, 0] = sbj
  frequencies[i, 1] = (whole_data['subject_id'] == sbj).sum()
cumolative_sums_of_samples = np.cumsum(frequencies[:,1])
frequencies[:, 2] = cumolative_sums_of_samples
frequencies[:, 3] = cumolative_sums_of_samples/cumolative_sums_of_samples[~0]

42654it [00:17, 2473.88it/s]


In [17]:
# visualizing the frequency  of subjects and images taken from each
vis_frequencies = pd.DataFrame(frequencies,columns=['subject_ID','n_of_sessions/images','cumolative_sessions','preccent_of_number_of_images'])
vis_frequencies.head(15)

,subject_ID,n_of_sessions/images,cumolative_sessions,preccent_of_number_of_images
0,17061176.0,2.0,2.0,0.000011
1,14108608.0,4.0,6.0,0.000033
2,17996251.0,17.0,23.0,0.000127
3,14522445.0,19.0,42.0,0.000233
4,13774104.0,1.0,43.0,0.000238
5,19797687.0,36.0,79.0,0.000437
6,19380289.0,2.0,81.0,0.000448
7,14410396.0,19.0,100.0,0.000554
8,16468805.0,5.0,105.0,0.000581
9,15087570.0,14.0,119.0,0.000659


In [18]:
# separating the testing split without having any mutual_subjects:
testing_subjects_masking = vis_frequencies['preccent_of_number_of_images'] < test_split
testing_subjects = vis_frequencies[testing_subjects_masking]['subject_ID'].values.tolist()
whole_train_subjects = vis_frequencies[~testing_subjects_masking]['subject_ID'].values.tolist()

In [19]:
# excluding irrelevant features
excluding2ds =  np.concatenate( (np.arange(32,41,dtype=int),
                                 np.arange(32,41,dtype=int)+ 116,
                                 np.arange(32,41,dtype=int)+116*2,
                                 np.arange(32,41,dtype=int)+116*3))

whole_data.drop(whole_data.columns[excluding2ds.tolist()],axis = 1,inplace=True)

In [20]:
# separating test_data and whole_trainig_data
whole_data = whole_data.sample(frac=1.0, random_state=random_seed)
Test = whole_data[whole_data['subject_id'].isin(testing_subjects)]
X_test, y_test = Test.iloc[:,:464-36].values, Test['new_race'].values

# separating test_data and whole_trainig_data
Train = whole_data[whole_data['subject_id'].isin(whole_train_subjects)]

# the size of training/testing_split:
print('The size of the testing split is: {}'.format(Test.shape))
print('The size of the whole trainig (train + valid) split is: {}'.format(Train.shape))

The size of the testing split is: (10833, 456)
The size of the whole trainig (train + valid) split is: (169771, 456)


In [21]:
# splitting the validation split:
X_train, X_val, y_train, y_val = train_test_split(Train.iloc[:,:464-36].values,
                                                  Train['new_race'].values,
                                                  test_size=valid_split,
                                                  random_state=random_seed
                                                  )

In [22]:
# checking the number of nans:
pd.isnull(X_train).sum(), pd.isnull(X_test).sum(), pd.isnull(X_val).sum()

(0, 0, 0)

In [23]:
# checking the number of classes in each split:
# train
(y_train==0).sum(),(y_train==1).sum(),(y_train==2).sum(), (y_val==0).sum(),(y_val==1).sum(),(y_val==2).sum()

(117993, 34800, 0, 13131, 3847, 0)

In [24]:
(y_test==0).sum(),(y_test==1).sum(),(y_test==2).sum()

(8484, 2349, 0)

In [25]:
print('The size of the validation split is: {}'.format(X_val.shape))
X_train.shape, X_val.shape, y_train.shape, y_val.shape

The size of the validation split is: (16978, 428)


((152793, 428), (16978, 428), (152793,), (16978,))

In [26]:
sm = SMOTE(random_state=21202)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [27]:
X_train.shape, y_train.shape

((235986, 428), (235986,))

In [28]:
# y_train  = pd.get_dummies(pd.DataFrame(y_train.tolist(),columns=['column_name']),columns=['column_name']).values * 1
# y_val  = pd.get_dummies(pd.DataFrame(y_val.tolist(),columns=['column_name']),columns=['column_name']).values * 1
# y_test  = pd.get_dummies(pd.DataFrame(y_test.tolist(),columns=['column_name']),columns=['column_name']).values * 1

In [29]:
# the number of samples after resampling:
X_train.shape, y_train.shape

((235986, 428), (235986,))

In [30]:
def classifer_1(features_number: int) -> callable:
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=100, activation="tanh", input_dim=features_number))
    model.add(keras.layers.Dropout(0.0))
    model.add(keras.layers.Dense(units=200, activation="tanh"))
    model.add(keras.layers.Dropout(0.0))
    model.add(keras.layers.Dense(units=1,activation='sigmoid'))
    model.summary()
    return model

def compiler_1(X_train_, Y_train_, X_valid_, Y_valid_,mdl_callback):
  initial_learning_rate = 0.0001
  epochs = 500
  batch_size = 512
  model = classifer_1(features_number=X_train_.shape[1])
  lss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
  # lss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
  metrics = ['accuracy',
                        tf.keras.metrics.AUC(curve='PR',name='PR-curve'),
                        tf.keras.metrics.AUC(curve='ROC',name='ROC-curve')]
  # metrics = metrics=[tf.keras.metrics.RootMeanSquaredError()]
  # metrics = [tf.keras.metrics.RootMeanSquaredError()]
  # lss = 'mean_absolute_error'
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-08),
                                                loss=lss,
                metrics=metrics)
  model.fit(x=X_train_,
            y=Y_train_,
            epochs=epochs,
            initial_epoch=0,
            batch_size=batch_size,
            validation_data=(X_valid_, Y_valid_),
            verbose=2,
            shuffle=True,
            callbacks=[mdl_callback])
  return model

In [43]:
# full Model
X_train_ = X_train.copy()
X_val_ = X_val.copy()
X_test_ = X_test.copy()

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

saving_path = '/content/gdrive/MyDrive/radiomics/second_results/mimic_race_' + 'full'+ '_'
# saving_path = 'chexpert_race_2_categ_' + 'full'+ '_'

belongings={'mean':mean_val,
            'feature_number':None,
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)
checkpoint_filepath = saving_path +'_mdl.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               42100     
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 200)               20200     
                                                                 
 dropout_5 (Dropout)         (None, 200)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                 201       
                                                                 
Total params: 62,501
Trainable params: 62,501
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500

Epoch 1: val_loss improved from inf t

KeyboardInterrupt: ignored

In [ ]:
# each segments:

segment_number = 1

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/second_results/mimic_race_' + '_' + segment_names[segment_number]+ '_'
# saving_path = 'chexpert_race_2_categ_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}
np.save(saving_path + 'belongings.npy',belongings)
checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_0 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               10600     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 200)               20200     
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 31,001
Trainable params: 31,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500

Epoch 1: val_loss improved from inf t

In [ ]:
# each segments:
segment_number = 1

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_race_2_categ_segment_' + segment_names[segment_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'
# checkpoint_filepath = '_mdl.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 100)               10600     
                                                                 
 dropout_14 (Dropout)        (None, 100)               0         
                                                                 
 dense_22 (Dense)            (None, 300)               30300     
                                                                 
 dropout_15 (Dropout)        (None, 300)               0         
                                                                 
 dense_23 (Dense)            (None, 3)                 903       
                                                                 
Total params: 41803 (163.29 KB)
Trainable params: 41803 (163.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [ ]:
# each segments:
segment_number = 2

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_2 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 100)               10600     
                                                                 
 dropout_16 (Dropout)        (None, 100)               0         
                                                                 
 dense_25 (Dense)            (None, 300)               30300     
                                                                 
 dropout_17 (Dropout)        (None, 300)               0         
                                                                 
 dense_26 (Dense)            (None, 3)                 903       
                                                                 
Total params: 41803 (163.29 KB)
Trainable params: 41803 (163.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [ ]:
# each segments:
segment_number = 3

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'

saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_3 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 100)               10600     
                                                                 
 dropout_14 (Dropout)        (None, 100)               0         
                                                                 
 dense_22 (Dense)            (None, 100)               10100     
                                                                 
 dropout_15 (Dropout)        (None, 100)               0         
                                                                 
 dense_23 (Dense)            (None, 1)                 101       
                                                                 
Total params: 20801 (81.25 KB)
Trainable params: 20801 (81.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epoch

In [ ]:
fos_features = (np.arange(0,18) + 0).tolist() + (np.arange(0,18) + 107).tolist() + (np.arange(0,18) + 107*2).tolist() + (np.arange(0,18) + 107*3).tolist()
_3d_features = (np.arange(18,32) + 0).tolist() + (np.arange(18,32) + 107).tolist() + (np.arange(18,32) + 107*2).tolist() + (np.arange(18,32) + 107*3).tolist()
glcm_features = (np.arange(32,56) + 0).tolist() + (np.arange(32,56) + 107).tolist() + (np.arange(32,56) + 107*2).tolist() + (np.arange(32,56) + 107*3).tolist()
glszm_features = (np.arange(56,72) + 0).tolist() + (np.arange(56,72)+ 107).tolist() + (np.arange(56,72) + 107*2).tolist() + (np.arange(56,72) + 107*3).tolist()
glrlm_features = (np.arange(72,88) + 0).tolist() + (np.arange(72,88)  + 107).tolist() + (np.arange(72,88)  + 107*2).tolist() + (np.arange(72,88)  + 107*3).tolist()
ngtdm_features = (np.arange(88,93) + 0).tolist() + (np.arange(88,93)  + 107).tolist() + (np.arange(88,93)  + 107*2).tolist() + (np.arange(88,93) + 107*3).tolist()
gldm_features = (np.arange(93,107) + 0).tolist() + (np.arange(93,107)  + 107).tolist() + (np.arange(93,107)  + 107*2).tolist() + (np.arange(93,107) + 107*3).tolist()

feature_names = ['fos_features', '_3d_features', 'glcm_features', 'glszm_features', 'glrlm_features', 'ngtdm_features', 'gldm_features']
feature_indexes = [fos_features, _3d_features, glcm_features, glszm_features, glrlm_features, ngtdm_features, gldm_features]

In [ ]:
feature_number = 0

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_0 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 1

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 2

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_2 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 3

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_3 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 4

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_4 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 5

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_5 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 6

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_6 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
# feature_number = 6

# X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
# X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
# X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

# mean_val = X_train_.mean(axis=0)
# std_val = X_train_.std(axis=0)

# # excluding invariant variables:
# mask_excluded = ~(std_val==0)
# X_train_[:,mask_excluded] -= mean_val[mask_excluded]
# X_train_[:,mask_excluded] /= std_val[mask_excluded]

# X_val_[:,mask_excluded] -= mean_val[mask_excluded]
# X_val_[:,mask_excluded] /= std_val[mask_excluded]

# X_test_[:,mask_excluded] -= mean_val[mask_excluded]
# X_test_[:,mask_excluded] /= std_val[mask_excluded]

# # excluding invariant variables:
# mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

# belongings={'mean':mean_val,
#             'feature_number':feature_names[feature_number],
#             'std':std_val,
#             'mask_excluded':mask_excluded,
#             'x_test':X_test_,
#             'y_test':y_test}

# np.save(saving_path + 'belongings.npy',belongings)

# checkpoint_filepath = saving_path +'_mdl.h5'

# model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
#                                                                 filepath=checkpoint_filepath,
#                                                                 save_weights_only=False,
#                                                                 monitor='val_loss',
#                                                                 save_best_only=True,
#                                                                 save_freq='epoch',
#                                                                 verbose=2
#                                                             )

# model_1_6 = compiler_1(X_train_ = X_train_[:,mask_excluded],
#                     Y_train_ = y_train,
#                     X_valid_ = X_val_[:,mask_excluded],
#                     Y_valid_ = y_val,
#                     mdl_callback=model_checkpoint_callback_1)